In [1]:
import numpy as np
import pandas as pd
# from google.colab.patches import cv2_imshow
from PIL import Image
import cv2
from pathlib import Path
import os
import json
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import torch

In [ ]:
# image = cv2.imread('train/4934.JPG')
# plt.figure(figsize = (10, 10))
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.show()

In [ ]:
data_df = pd.read_csv("train.csv")

In [ ]:
# data_df.count_region.value_counts()

In [ ]:
# data_df.head(3)

In [ ]:
people_in_pic = data_df[data_df['count_region'] > 0]

In [ ]:
people_in_pic = people_in_pic.reset_index(drop=True)

In [ ]:
people_in_pic['region_shape'][0]

In [2]:
import sys
import random

In [ ]:
vowels = ('[',']',"'",':','c','x','y','r','{','}','"', ' ')
for i in tqdm(range(len(people_in_pic))):
    path_img = people_in_pic['ID_img'][i].split('.')
    image = cv2.imread(f'train/{path_img[0]}.JPG')
    cv2.imwrite(f'data/images/{path_img[0]}.JPG', image)
    y_img, x_img, _ = image.shape
#     print(x_img, y_img, path_img)
    count_people = people_in_pic['count_region'][i]
    coor_people = people_in_pic['region_shape'][i]
    l_1 = ''.join([c for c in coor_people if c not in vowels])
    l = l_1.split(',')
    coordinates = []
    for j in range(int(count_people)):
        
#         x1 = int(l[0+j*3])- int(float(l[2+j*3])/2)
#         y1 = int(l[1+j*3]) - int(float(l[2+j*3])/2)
#         x2 = int(float(l[2+j*3])/2) + int(l[0+j*3])
#         y2 = int(float(l[2+j*3])/2) + int(l[1+j*3])
# #         print(x1,y1,x2,y2)
#         cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,255), 10)
        x = int(l[0+j*3])/x_img
        y = int(l[1+j*3])/y_img
        w = float(l[2+j*3]) / x_img
        h = float(l[2+j*3]) / y_img
#         print(int(l[0+j*3]),int(l[1+j*3]),int(l[2+j*3]))
        coordinates.append('0' + ' '+ str(float('{:.6f}'.format(x))) + ' '  + str(float('{:.6f}'.format(y))) + ' '  + str(float('{:.6f}'.format(w))) + ' '  + str(float('{:.6f}'.format(h))))
#     print(coordinates)
    
    with open(f'data/labels/{path_img[0]}.txt', 'w') as f:#os.path.join(labels, Path(id).with_suffix('.txt')), 'a') as f:
        for i in range(int(count_people)):
            f.write(coordinates[i] + '\n')#f"{cls} {' '.join(f'{x:.6f}' for x in box[0])}\n")
    f.close()
#     break

In [ ]:
# # image = cv2.imread('train/4934.JPG')
# plt.figure(figsize = (10, 10))
# plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
# plt.show()

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5 && pip install -r requirements.txt

In [ ]:
sys.path.append('yolov5')

In [24]:
import gc

gc.collect()

2348

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!cd yolov5 && python train.py --img 1024 --batch 2 --epochs 50 --data data.yaml --weights yolov5m.pt --cache

In [ ]:
!cd yolov5 && python detect.py --img 1024 --weights C:/Users/NDA/PycharmProjects/omsk/1024_300.pt --conf 0.1 --source C:/Users/NDA/PycharmProjects/omsk/data/test

In [25]:
import torch

In [26]:
model_path = '2048_500.pt'
yolo_path = 'yolov5'
model = torch.hub.load(yolo_path, 'custom', path=model_path, source='local')
model.conf = 0.1  # NMS confidence threshold
# model.iou = 0.45  # NMS IoU threshold
#       agnostic = False  # NMS class-agnostic
#       multi_label = False  # NMS multiple labels per box
#       classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
#       max_det = 1000  # maximum number of detections per image
#       amp = False  # Automatic Mixed Precision (AMP) inference


YOLOv5  v6.2-233-g892c4cd Python-3.9.13 torch-1.12.1+cpu CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [27]:
df_test = pd.read_csv("test.csv")

In [20]:
df_test.head(20)


,ID_img,region_shape
0,1.JPG,0
1,2.JPG,0
2,3.JPG,0
3,4.JPG,0
4,5.JPG,0
5,6.JPG,0
6,7.JPG,0
7,8.JPG,0
8,9.JPG,0
9,10.JPG,0


In [28]:
for j in tqdm(range(len(df_test))):
    img_path = f"test/test/{df_test['ID_img'][j]}"
    image = cv2.imread(img_path)
    y, x, _ = image.shape
    results = model([img_path], size=2048)
    coord_pred = []
    if results.xyxy[0].shape != torch.Size([0, 6]):
        for i in range(len(results.xyxy[0])):
            x_1 = int(float(results.xyxy[0][i][0]))
            y_1 = int(float(results.xyxy[0][i][1]))
            x_2 = int(float(results.xyxy[0][i][2]))
            y_2 = int(float(results.xyxy[0][i][3]))
            r = max(x_2-x_1, y_2-y_1)
            cx = x_1 + int(r/2)
            cy = y_1 + int(r/2)
            t = '{"cx":' +str(cx)+', "cy":'+str(cy)+ ', "r":' +str(r)+'}'
            if r > 30:
                coord_pred.append(t)        
        sort_coord = sorted(coord_pred)
#         df_test.at[j, 'region_shape']  = sort_coord
        if len(sort_coord) > 0:
            df_test.loc[:, 'region_shape'].loc[j] = sort_coord


C:\Users\NDA\AppData\Local\Temp\ipykernel_15060\758794589.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j in tqdm(range(len(df_test))):


  0%|          | 0/3435 [00:00<?, ?it/s]

C:\Users\NDA\AppData\Local\Temp\ipykernel_15060\758794589.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.loc[:, 'region_shape'].loc[j] = sort_coord


In [29]:
df_test

,ID_img,region_shape
0,1.JPG,0
1,2.JPG,0
2,3.JPG,0
3,4.JPG,0
4,5.JPG,0
...,...,...
3430,3431.JPG,0
3431,3432.JPG,0
3432,3433.JPG,0
3433,3434.JPG,0


In [30]:
df_test.to_csv("submit1_2048.csv", index=False) #sample_solution_df.to_csv('sample_solution_27-08-22.csv', sep=';', index=False)